In [5]:
import tqdm

import numpy as np

import src

In [6]:
train_models = src.shapenet.get_shapenet_data("train")
clouds = []
for i in tqdm.tqdm(range(20)):
    model_path = train_models[np.random.randint(len(train_models))]
    cloud = src.shapenet.sample_points(model_path, nb=100000)
    clouds.append(cloud)

100%|██████████| 20/20 [00:01<00:00, 12.71it/s]


In [7]:
extracted = []
extractor = src.dataset.PPFPatchExtractor(nb_patches=2044, nb_points=1024)
for cloud in tqdm.tqdm(clouds):
    ppf, M = extractor.make_example(cloud)
    extracted.append((ppf,M))

100%|██████████| 20/20 [00:22<00:00,  1.13s/it]


In [8]:
loss_func = src.loss.N_tuple_loss(extractor.nb_patches)
for ppf, M in tqdm.tqdm(extracted):
    pred = np.random.random((1, extractor.nb_patches, 13)).astype(np.float32)
    res = loss_func(M[np.newaxis,...],pred)

100%|██████████| 20/20 [00:00<00:00, 92.11it/s]


In [9]:
ppf.shape

(2044, 1024, 10)